In [ ]:
# Open Questions (maybe ask Ricardo):
# - can we remove hasDamage before the actual feature selection or is it only allowed to remove features in the FS part even though this would lead to unnecessary steps and wasting computing power

**Your work will be evaluated according to the following criteria:**
- Project Structure and Notebook(s) Quality (4/20)
- Data Exploration & Initial Preprocessing (4/20)
- Regression Benchmarking and Optimization (7/20)
- Open-Ended Section (4/20)
- Deployment (1/20)
- Extra Point: Have Project Be Publicly Available on GitHub (1/20)


**Project Timeline**
- 22.11.: Preprocessing and Model Preparation
    - Finish clean preprocessing all included in pipeline
    - Finish clean Hyperparameter Tuning
- 29.11.: Feature Selection
    - Clean and structured approach for feature selection for all models (best case: consistent approach imo)
- 29.11.: Regression Benchmarking and Optimization
    - Automize Optimization (add something like mlflow)
- 06.12.: Open-End Section and Deployment
    - Added 4 open-end-experiments
    - Deployment
- 13.12.: Notebook Feinschliff
    - Super clean notebook structure similar to lab-notebooks by Ricardo
    - Show and explain results of different models clearly in markdown tables etc. (see the lab-notebooks)
- 14.12.: Submission

In [ ]:
# TODO Open End Section:
# Interface for new Car

<div style="
    background: rgba(25, 25, 25, 0.55);
    backdrop-filter: blur(16px) saturate(150%);
    -webkit-backdrop-filter: blur(16px) saturate(150%);
    border: 1px solid rgba(255, 255, 255, 0.12);
    border-radius: 18px;
    padding: 45px 30px;
    text-align: center;
    font-family: 'Inter', 'Segoe UI', 'Helvetica Neue', Arial, sans-serif;
    color: #e0e0e0;
    box-shadow: 0 0 30px rgba(0, 0, 0, 0.35);
    margin: 40px auto;
    max-width: 800px;
">

  <h1 style="
      font-size: 2.8em;
      font-weight: 700;
      margin: 0 0 8px 0;
      letter-spacing: -0.02em;
      background: linear-gradient(90deg, #00e0ff, #9c7eff);
      -webkit-background-clip: text;
      -webkit-text-fill-color: transparent;
  ">
      Machine Learning Project
  </h1>

  <h2 style="
      font-size: 1.6em;
      font-weight: 500;
      margin: 0 0 25px 0;
      color: #b0b0b0;
      letter-spacing: 0.5px;
  ">
      Cars 4 You - Predicting Car Prices
  </h2>

  <p style="
      font-size: 1.25em;
      font-weight: 500;
      color: #c0c0c0;
      margin-bottom: 6px;
  ">
      Group 5 - Lukas Belser, Samuel Braun, Elias Karle, Jan Thier
  </p>

  <p style="
      font-size: 1.05em;
      font-weight: 400;
      color: #8a8a8a;
      font-style: italic;
      letter-spacing: 0.5px;
  ">
      Machine Learning End Results · 22.12.2025
  </p>
</div>


### **Table of Contents**
 
- [1. Import Packages and Data](#1-import-packages-and-data)  
  - [1.1 Import Required Packages](#11-import-required-packages)  
  - [1.2 Load Datasets](#12-load-datasets)  
  - [1.3 Kaggle Setup](#13-kaggle-setup)  
- [2. Data Cleaning, Feature Engineering, Split & Preprocessing](#2-data-cleaning-feature-engineering-split--preprocessing)  
  - [2.1 Data Cleaning](#21-data-cleaning)  
  - [2.2 Feature Engineering](#22-feature-engineering)  
  - [2.3 Data Split](#23-data-split)  
  - [2.4 Preprocessing](#24-preprocessing)  
- [3. Feature Selection](#3-feature-selection)  
- [4. Model Evaluation Metrics, Baselining, Setup](#4-model-evaluation-metrics-baselining-setup)  
- [5. Hyperparameter Tuning and Model Evaluation](#5-hyperparameter-tuning-and-model-evaluation)  
  - [5.1 ElasticNet](#51-elasticnet)  
  - [5.2 HistGradientBoost](#52-histgradientboost)  
  - [5.3 RandomForest](#53-randomforest)  
  - [5.4 ExtraTrees](#54-extratrees)  
- [6. Feature Importance of Tree Models (with SHAP)](#6-feature-importance-of-tree-models-with-shap)  
  - [6.1 HGB](#61-hgb)  
  - [6.2 RF](#62-rf)  
- [7. Kaggle Competition](#7-kaggle-competition)  

TODO finish + update toc > at the end of project

### 1. Import Packages and Data

#### 1.1 Import Required Packages

In [ ]:
!pip install kaggle
!pip install shap
!pip install -U scikit-learn
!pip install category_encoders
!pip install ydata-profiling

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt; plt.rcParams.update({"figure.max_open_warning": 0, "figure.dpi": 100})
import joblib
import shap

from collections import Counter
from sklearn.feature_selection import VarianceThreshold, RFE
from scipy.stats import spearmanr, uniform, randint
from sklearn.metrics import mean_absolute_error
 
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, TargetEncoder, StandardScaler, FunctionTransformer, RobustScaler
from sklearn.base import clone, BaseEstimator, TransformerMixin
 
from sklearn.model_selection import RandomizedSearchCV, KFold, GridSearchCV, cross_validate
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import GradientBoostingRegressor, HistGradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, StackingRegressor
from sklearn.svm import SVR
from sklearn.inspection import permutation_importance
from sklearn.dummy import DummyRegressor
from sklearn.feature_selection import SelectFromModel


from tqdm.auto import tqdm

from category_encoders import QuantileEncoder # used for median target encoding (sklearn only supports mean target encoding with their TargetEncoder class)
 
from car_functions import clean_car_dataframe
from pipeline_functions import GroupImputer, m_estimate_mean, CarFeatureEngineer, NamedFunctionTransformer, get_feature_names_from_preprocessor, to_float_array, model_hyperparameter_tuning, DebugTransformer

from collections import Counter
from sklearn.inspection import permutation_importance
from tqdm.auto import tqdm

#### 1.2 Load Datasets

In [ ]:
df_cars_train = pd.read_csv("train.csv")
df_cars_test = pd.read_csv("test.csv")

#### 1.3 Kaggle Setup

In [ ]:
# Kaggle API Connect

# Folder containing kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = "/Workspace/Users/20250355@novaims.unl.pt" #add your own kaggle.json api token

# Test
!echo $KAGGLE_CONFIG_DIR

### 2. Preprocessing

#### 2.1 Data Cleaning

In [ ]:
# Outlier Preprocessing, Missing Value Handling and Decision justifying happens here
# TODO add clean_car_dataframe to pipeline
df_cars_train = clean_car_dataframe(df_cars_train)
df_cars_test = clean_car_dataframe(df_cars_test)

# Safety Check: print unique values of all columns of df_cars_train // df_cars_test to see if data cleaning worked and if there are still odd values
for col in df_cars_train.columns:
    print(col, df_cars_train[col].unique())
print("X"*150)
for col in df_cars_test.columns:
    print(col, df_cars_test[col].unique())

#### 2.2 (No) Data Split

**Our approach:**
- Train: use cross-validation in the sklearn pipeline on the training data
- Test: Use external hold-out set from kaggle as final test set    
-> An additional val set is therefore not necessary and would waste training data


<u>Place in the pipe:</u> The split is decided here because the data has to be split before all of the following preprocessing steps to avoid data leakage

In [ ]:
X_train = df_cars_train.drop(columns='price')
y_train = df_cars_train['price']

#### 2.3 Handling missing values

**Our approach:**
- `Group Imputer`: We use a custom GroupImputer that imputes the missing values to be the median of entries within the same group
    - For that we use a hierarchical structure to identify the most similar group to the one with the missing value:
        - 1st level: ...
        - 2nd level: ...
        - ...
        - 4th level: Model
        - 5th level: Brand

<u>Place in the pipe:</u> The Imputation is decided here because the data has to be imputed on original values before engineering new features

#### 2.4 Feature Engineering

**Our approach:**
- We use the class CarFeatureEngineer to be able create the engineered features within the pipeline to prevent data leakage

<u>Base Feature Creation</u>

These are foundational features derived directly from the original data, often to create linear relationships or capture interactions.
- `age`: Calculated as `2020 - year`. Creates a simple linear feature representing the car's age. Newer cars (lower age) generally have higher prices.
- `miles_per_year`: Calculated as `mileage / age`. This normalizes the car's usage, preventing high correlation (multicollinearity) between `mileage` and `age`. A 3-year-old car with 60,000 miles is different from a 6-year-old car with 60,000 miles.
- `age_x_engine`: An interaction term `age * engineSize`. This helps the model capture non-linear relationships, such as the possibility that the value of cars with large engines might depreciate faster (or slower) than cars with small engines.
- `mpg_x_engine`: An interaction term `mpg * engineSize`. This captures the combined effect of fuel efficiency and engine power.
- `tax_per_engine`: Calculated as `tax / engineSize`. This feature represents the tax cost relative to the engine's power, which could be an indicator of overall running costs or vehicle class.
- `mpg_per_engine`: Calculated as `mpg / engineSize`. This creates an "efficiency" metric, representing how many miles per gallon the car achieves for each unit of engine size.


<u>Popularity & Demand Features</u>

These features attempt to quantify a car's popularity or market demand, which directly influences price.
- `model_freq`: Calculates the frequency (percentage) of each `model` in the training dataset. Popular, common models often have more stable and predictable pricing and demand.


<u>Price Anchor Features</u>

These features "anchor" a car's price relative to its group. They provide a strong baseline price signal based on brand, model, and configuration.
- `brand_med_price`: The median price for the car's `Brand` (e.g., the typical price for a BMW vs. a Skoda). This captures overall brand positioning.
- `model_med_price`: The median price for the car's `model` (e.g., the typical price for a 3-Series vs. a 1-Series). This captures the model's positioning within the brand.
- `brand_fuel_med_price`: The median price for the car's specific `Brand` and `fuelType` combination (e.g., a Diesel BMW vs. a Petrol BMW).
- `brand_trans_med_price`: The median price for the `Brand` and `transmission` combination (e.g., an Automatic BMW vs. a Manual BMW).


<u>Normalized & Relative Features</u>

These features compare a car to its peers rather than using absolute values.
- `*_anchor` (e.g., `brand_med_price_anchor`): Created by dividing the median price features (from section 3) by the `overall_mean_price`. This makes the feature dimensionless and represents the group's price *relative* to the entire market (e.g., "this brand is 1.5x the market average").
- `age_rel_brand`: Calculated as `age - brand_median_age`. This shows if a car is newer or older than the *typical* car for that specific brand, capturing relative age within its own group.


<u>CV-Safe Target Encodings</u>

This is an advanced technique to encode categorical variables (like `model` or `Brand`) using information from the target variable (`price`) without causing data leakage.
- `*_te` (e.g., `model_te`): Represents the *average price* for that category (e.g., the average price for a "Fiesta").
- **Why is it "CV-Safe"?** Instead of just calculating the global average price for "Fiesta" and applying it to all rows (which leaks target information), this method uses K-Fold cross-validation. For each fold of the data, the target encoding is calculated *only* from the *other* folds. This ensures the encoding for any given row never includes its own price, preventing leakage and leading to a more robust model.

#### 2.5 Outlier Handling

In [ ]:
# TODO
# e.g. how to handle Zeros in tax (use groupimputer?) -> features that are computed with tax are also affected and need to be handled then ~J

#### 2.6 Encoding, Transforming and Scaling

**Our approach:**
- We identify different treatments for different groups of variables and combine all of them in the ColumnTransformer

In [ ]:
# Original features
orig_numeric_features = ["year", "mileage", "tax", "mpg", "engineSize", "previousOwners", "hasDamage"]
# TODO create origic_boolean_features for hasDamage ~J
orig_categorical_features = ["Brand", "model", "transmission", "fuelType"]

##### [2.6.0 Baseline Pipe]

**Our approach:**
- We create two different pipelines to compare our fully adjusted model to the baseline
    - original features with simple imputer
    - engineered features with group imputer

In [ ]:
# PIPELINE WITH preprocessor_orig CONTAINING ONLY ORIGINAL FEATURES and USING SIMPLE IMPUTATION
numeric_transformer_orig = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),    # simple global median imputation
    ("to_float", FunctionTransformer()), # TODO dont we have to add the to_float_array function here too? ~J
    ("scaler", RobustScaler())
])

categorical_transformer_orig = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),  # fill by mode instead of Unknown
    ("encoder", OneHotEncoder(handle_unknown="ignore"))  # One-hot encoding
])

preprocessor_orig = ColumnTransformer([
    ("num", numeric_transformer_orig, orig_numeric_features),
    ("cat", categorical_transformer_orig, orig_categorical_features)
])

##### 2.6.1 Categorize Features

**Our approach:**
- Numeric Features:
    - For scaling:
        - ...
    - For log-transformation before scaling (due to right-skew identified in EDA):
        - ...
- Categorical Features:
    - For OHE:
        - ...
    - For TE:
        - ...
- Unused Features:
    - `year`: dropped because replaced by derived feature 'age'
    - `hasDamage`: droppped because unclear what 0 and NaN mean
    - `paintQuality`: dropped because added by mechanic so not available for our predictions in production

In [ ]:
# TODO maybe this step can also be automated within the pipeline ~J
numeric_features = [
    "hasDamage",
    "age", "tax", "mpg", "engineSize", "previousOwners",        # Original features (mileage is handled separately because of log transformation)
    "mpg_x_engine", "engine_x_age", "mpg_x_age", "tax_x_age",   # multiplication interaction features (multiplying for amplification)                                   
    "mpg_per_engine", "tax_per_mpg",                            # division interaction features (division for normalization for ratios (efficiency))                 
    "model_freq",
    "age_rel_brand",
]
numeric_features_for_log = ["mileage", "miles_per_year"] #, "mileage_x_age"] # mileage_x_age decreases performance slightly
boolean_features = ["hasDamage"]                                # TODO create logic for boolean features in GroupImputer and ColumnTransformer
categorical_features_ohe = ["transmission", "fuelType"]
categorical_features_te_mean = ["Brand", "model"]
categorical_features_te_median = ["Brand", "model",             # original features
                                  "brand_fuel", "brand_trans"]  # engineered features for anchors
unused_columns = ["year"]                                       # replaced by age

##### 2.6.2 Create Pipelines for each feature type

**Our approach:**
- `Transformation` for skewed Numerics:
    - Log-transform for right-skewed variables
    - box-cox not used in final pipe because ...
- `Scaler` for Numerics:
    - StandardScaler because ...
    - MinMaxScaler performed worse because ...
    - RobustScaler performed worse because ...
- `Encoding` for Categoricals:
    - Low cardinality:
        - OHE because best performance with tree-based models
    - High Cardinality:
        - Median TE on categorical features because performs better than Mean TE
- `ColumnTransformer`: All operations are combined in the ColumnTransformer which applies the different steps to different columns of the data in one unified pipeline (reproducible and prevents data leakage)    
  -> outputs a combined feature matrix

In [ ]:
# TODO two different scalers are used here. Isnt it better to use exactly one? ~J 
log_transformer_and_scaler = Pipeline([
    # Hierarchical imputation on Brand_te/model_te, then log-transform
    ("to_float", NamedFunctionTransformer(to_float_array, validate=False)),
    ("log",    NamedFunctionTransformer(np.log1p, validate=False)),  # log1p handles zeros safely
    ("scaler", RobustScaler()),
])

numeric_scaler = Pipeline([
    ("to_float", NamedFunctionTransformer(to_float_array, validate=False)),
    ("scaler", RobustScaler()),
])

categorical_transformer_ohe = Pipeline([
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
])

# Keep mean target encoder in the code but dont use it for now because median TE seems more robust and we use only one method for consistency ~J
# categorical_transformer_te_mean = Pipeline([ 
#     ("encoder", TargetEncoder(target_type='continuous', cv=5, smooth='auto', random_state=42)), # Prevents data leakage with CV (e.g. for the samples in Fold 1, it calculates the target mean using the data from Folds 2, 3, 4, and 5) # TODO If it overfits test data too much, increasing the smoothing parameter can help
#     ("scaler", StandardScaler()),
# ])

# Names for median-TE features (one per input column, since QuantileEncoder outputs 1 column per feature)
median_te_feature_names = [f"{col}_median_te" for col in categorical_features_te_median]
categorical_transformer_te_median = Pipeline(steps=[
    ('median_encoder', QuantileEncoder(quantile=0.5, m=10.0)), # not specifying the cols means it encodes all columns (m is the smoothing parameter -> smoothing mitigates but doesnt eliminate leakage) # TODO tune m?
    # TODO Why is to_float not needed here but in other pipelines? ~J
    ('scaler', RobustScaler()) ,
    ('name_wrapper', NamedFunctionTransformer(feature_names=median_te_feature_names,
                                              validate=False)),
])

enc_transf_scale = ColumnTransformer([
    ("log", log_transformer_and_scaler, numeric_features_for_log),
    ("num", numeric_scaler, numeric_features),
    ("cat", categorical_transformer_ohe, categorical_features_ohe),
    # ("mean_te", categorical_transformer_te_mean, categorical_features_te_mean), # Mean TE is currently not used but we keep it in the code for reference or later experimenting ~J
    ("median_te", categorical_transformer_te_median, categorical_features_te_median)
])

##### 2.7 Feature Selection

For Feature Selection Strategy, we first designed a domain-specific preprocessing pipeline (feature engineering + group-wise imputation + encoding).

We use:
1. **Filter**: VarianceThreshold() to remove constant features.
2. **Embedded**: SelectFromModel(RandomForest) inside each model pipeline as a supervised feature selector.
    - This selector is trained within cross-validation and shared across all models, ensuring: _No data leakage, Consistent feature selection logic, Model-agnostic, non-linear evaluation of feature relevance._
    - We only use this supervised selector for our models that are more sensitive to high dimensionality and collinearity (ElasticNet, SVR)

For tree-based ensemble models (RF, ET, HGB, and SR), our runs showed that explicit feature selection did **not improve** performance, as these models already handle redundant features well through mechanisms like greedy node splitting, feature subsampling (bagging), and iterative error correction.

Therefore, for these models we only use VarianceThreshold filter.


-----

SKlearn elements we also considered:
- Filter Methods: SelectKBest & SelectPercentile
- Embedded: No more than SelectFromModel
- Wrapper Methods: RFE, RFECV, SequentialFeatureSelector (too expensive)

In [ ]:
# TODO Samuel further review SKlearn pipeline FS techniques and see how to improve MAE again

# Base estimator used purely for feature selection
fs_estimator = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    bootstrap=True,
    n_jobs=-1,
    random_state=42,
)

# TODO multiple techniques and then majority voting ~J

# Global feature selector, used in *all* pipelines # TODO this was not used anywhere yet so i commented it. Do you want to apply it somewhere or why is that still here @Samu? ~J
# feature_selector = SelectFromModel(
#     estimator=fs_estimator,
#     threshold="median",   # keep features with importance above median - tuning with mean or numeric values
#     prefit=False,         # fit inside the pipeline
# )

# TODO VarianceThreshold has to be applied before scaling while most of the other FS techniques should be applied after scaling ~J
fs_pipe_remove_redundant = Pipeline([
    ("vt", VarianceThreshold(threshold=0.0)),
    ("fs", SelectFromModel(fs_estimator, threshold="median")),  # shared selector
])

fs_pipe_keep_redundant = Pipeline([
    ("vt", VarianceThreshold(threshold=0.0))
])

#### 2.9 Create Final Preprocessing Pipeline

**Our approach:**
- The `Pipeline` combines feature engineering, group imputation and the column transformer into the final preprocessing pipe
    - Feature Engineering: see Markdown in Section 2.2
    - Group Imputer: see Markdown in ...
    - Column Transformer: see explanation in line 1

In [ ]:
# Preprocessor with feature engineering, group imputation, and ColumnTransformer
preprocessor_pipe = Pipeline([
    ("group_imputer", GroupImputer(
        group_cols=("Brand", "model"),
        num_cols=orig_numeric_features, # numeric_features + numeric_features_for_log,                      # We have to use the original features here because the others are engineered in the next step
        cat_cols=orig_categorical_features, # categorical_features_ohe + categorical_features_te_median,    # We have to use the original features here because the others are engineered in the next step
        fallback="__MISSING__",
    )),
    ("fe", CarFeatureEngineer(ref_year=2020)),
    ("ct", enc_transf_scale),
    # TODO add fs here 
])

In [ ]:
# Preprocessor with feature engineering, group imputation, and ColumnTransformer
show_data = True # needs to be set to False when running the models because 'display' is used
y_data_profiling = True
debug_preprocessor_pipe = Pipeline([
    ('debug_start', DebugTransformer('START', show_data=show_data, y_data_profiling=y_data_profiling)),
    ("group_imputer", GroupImputer(
        group_cols=("Brand", "model"),
        num_cols=orig_numeric_features, # numeric_features + numeric_features_for_log,                      # We have to use the original features here because the others are engineered in the next step
        cat_cols=orig_categorical_features, # categorical_features_ohe + categorical_features_te_median,    # We have to use the original features here because the others are engineered in the next step
        fallback="__MISSING__",
    )),
    ('debug_after_impute', DebugTransformer('AFTER IMPUTATION', show_data=show_data, y_data_profiling=y_data_profiling)),
    ("fe", CarFeatureEngineer(ref_year=2020)),
    ('debug_after_fe', DebugTransformer('AFTER FEATURE ENGINEERING', show_data=show_data, y_data_profiling=y_data_profiling)),
    ("ct", enc_transf_scale),
    ('debug_end', DebugTransformer('END', show_data=show_data, y_data_profiling=y_data_profiling))
    # TODO add fs here 
])

print("Show outputs of each step in the preprocessing pipeline:") # Set show_data=True in DebugTransformer to see the data at each step
X_result = debug_preprocessor_pipe.fit_transform(X_train, y_train)

In [ ]:
# # TODO delete following cell later - this is for us to see if the group imputer works - but it is GPT slop

# brand = "VW"
# model = "golf"

# # 1) Get the fitted steps from preprocessor_pipe
# preprocessor_pipe.fit(X_train, y_train)
# fe = preprocessor_pipe.named_steps["fe"]              # CarFeatureEngineer
# imp = preprocessor_pipe.named_steps["group_imputer"]  # GroupImputer

# # 2) Inspect GroupImputer internal numeric stats
# pair_table = getattr(imp, "num_pair_", None)    # indexed by (_g0, _g1) = (Brand, model)
# brand_table = getattr(imp, "num_first_", None)  # indexed by _g0 = Brand
# global_med = getattr(imp, "num_global_", None)  # Series of global medians

# print("Has pair-level medians table:",
#       pair_table is not None and not getattr(pair_table, "empty", True))
# print("Has brand-level medians table:",
#       brand_table is not None and not getattr(brand_table, "empty", True))
# print("Has global median:",
#       global_med is not None and not global_med.empty if global_med is not None else False)
# print()

# _g0 = brand
# _g1 = model

# # 2a) Pair-level
# if pair_table is not None and (_g0, _g1) in pair_table.index:
#     print(f"Pair-level median FOUND for ({brand}, {model}):")
#     display(pair_table.loc[(_g0, _g1)])
# else:
#     print(f"No pair-level median for ({brand}, {model}).")
#     if pair_table is not None and not pair_table.empty:
#         print("Sample of pair-level medians (top 5):")
#         display(pair_table.head())

# # 2b) Brand-level
# if brand_table is not None and _g0 in brand_table.index:
#     print(f"\nBrand-level median for {brand}:")
#     display(brand_table.loc[_g0])
# else:
#     print("\nNo brand-level median for", brand)
#     if brand_table is not None and not brand_table.empty:
#         print("Sample of brand-level medians (top 5):")
#         display(brand_table.head())

# # 2c) Global medians
# print("\nGlobal median (fallback):")
# display(global_med)

# # 3) Apply CarFeatureEngineer + GroupImputer to VW Golf rows and compare
# #    (GroupImputer was fitted after CarFeatureEngineer, so we must mimic that order)

# # 3a) Feature engineering on full X_train
# X_train_fe = fe.transform(X_train)

# # 3b) Filter for VW Golf in the feature-engineered space
# vw_golf = X_train_fe[(X_train_fe["Brand"] == brand) & (X_train_fe["model"] == model)].copy()

# if vw_golf.empty:
#     print("\nNo VW Golf rows found in X_train.")
# else:
#     print(f"\nFound {len(vw_golf)} VW Golf rows in X_train.")

#     # 3c) GroupImputer expects the columns it saw at fit time
#     cols_for_imp = imp.feature_names_in_
#     vw_input = vw_golf.loc[:, cols_for_imp]

#     vw_imp = imp.transform(vw_input)
#     vw_imp_df = pd.DataFrame(vw_imp, columns=cols_for_imp, index=vw_golf.index)

#     print("\nImputed data (first 8 rows):")
#     display(vw_imp_df[["mpg", "mileage", "tax"]].head(8))

#     # 4) Build comparison table (original vs imputed, for selected columns)
#     comp = pd.DataFrame(index=vw_golf.index)
#     comp["orig_mpg"] = vw_golf["mpg"]
#     comp["imp_mpg"] = vw_imp_df["mpg"]
#     comp["orig_tax"] = vw_golf["tax"]
#     comp["imp_tax"] = vw_imp_df["tax"]
#     comp["orig_mileage"] = vw_golf["mileage"]
#     comp["imp_mileage"] = vw_imp_df["mileage"]

#     print("\nOriginal vs imputed (first 12 rows):")
#     display(comp.head(12))

#     # 5) Determine imputation source per row
#     def source_of_imputation(col):
#         srcs = []
#         for idx, row in comp.iterrows():
#             val = row[f"imp_{col}"]
#             src = "other"

#             # Pair-level
#             if pair_table is not None and (_g0, _g1) in pair_table.index and col in pair_table.columns:
#                 pair_val = pair_table.loc[(_g0, _g1), col]
#                 if pd.notna(pair_val) and pd.notna(val) and val == pair_val:
#                     src = "pair"

#             # Brand-level
#             if src == "other" and brand_table is not None and _g0 in brand_table.index and col in brand_table.columns:
#                 brand_val = brand_table.loc[_g0, col]
#                 if pd.notna(brand_val) and pd.notna(val) and val == brand_val:
#                     src = "brand"

#             # Global
#             if src == "other" and global_med is not None and col in global_med.index:
#                 glob_val = global_med[col]
#                 if pd.notna(glob_val) and pd.notna(val) and val == glob_val:
#                     src = "global"

#             srcs.append(src)
#         return srcs

#     comp["src_mpg"] = source_of_imputation("mpg")
#     comp["src_tax"] = source_of_imputation("tax")
#     comp["src_mileage"] = source_of_imputation("mileage")

#     print("\nImputation sources for the shown rows:")
#     display(comp.head(12))

#     # 6) Summary counts: NaN before vs after imputation
#     print("\nSummary counts: NaN before -> NaN after")
#     before = vw_golf[["mpg", "mileage", "tax"]].isna().sum()
#     after = pd.DataFrame({
#         "mpg": comp["imp_mpg"],
#         "mileage": comp["imp_mileage"],
#         "tax": comp["imp_tax"],
#     }).isna().sum()
#     display(pd.DataFrame({"na_before": before, "na_after": after}))

### 4. Model Evaluation Metrics, Baselining, Setup

#### 4.1 Model Evaluation Metrics

**MAE (Mean Absolute Error):**
- average absolute deviation between predicted and true car prices
- easy to interpret in pounds, same metric used by Kaggle competition

**RMSE (Root Mean Squared Error):**
- sensitive to outliers, helps identify large prediction errors

**R²:**
- Coefficient of determination: proportion of variance explained by the model
- 1.0 = perfect predictions, 0.0 = same as predicting mean, < 0.0 = worse than mean

#### 4.2 Baseline (median)

In [ ]:
# Baseline: DummyRegressor using the median price as prediction
baseline_pipe = Pipeline([
    ("preprocess", preprocessor_pipe),  
    ("model", DummyRegressor(strategy="median")),
])

baseline_cv = cross_validate(
    baseline_pipe,
    X_train,
    y_train,
    cv=3,
    scoring={
        "neg_mae": "neg_mean_absolute_error",
        "neg_mse": "neg_mean_squared_error",
        "r2": "r2",
    },
    n_jobs=-2,
    verbose=0,
)

baseline_mae = -baseline_cv["test_neg_mae"].mean()
baseline_rmse = np.sqrt(-baseline_cv["test_neg_mse"].mean())
baseline_r2 = baseline_cv["test_r2"].mean()

print("Baseline (median) performance on CV:")
print(f"MAE:  {baseline_mae:,.4f}")
print(f"RMSE: {baseline_rmse:,.4f}")
print(f"R2:   {baseline_r2:,.4f}")

# Baseline (median) with engineered features & CV:
# MAE:  6,801.8131
# RMSE: 9,981.0186
# R2:   -0.0508

#### 4.3 Pipeline Definitions (preprocessor + model)

##### 4.3.1 Using Baseline Preprocessing

In [ ]:
elastic_pipe_orig = Pipeline([
    ("preprocess", preprocessor_orig),
    ("model", ElasticNet(
        alpha=0.01,
        l1_ratio=0.5,
        max_iter=30000,
        tol=1e-4,
        selection="cyclic",
        random_state=42,
    )),
])

hgb_pipe_orig = Pipeline([
    ("preprocess", preprocessor_orig),
    ("model", HistGradientBoostingRegressor(
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=20,
        l2_regularization=0.5,
        random_state=42,
    )),
])

rf_pipe_orig = Pipeline([
    ("preprocess", preprocessor_orig),
    ("model", RandomForestRegressor(
        n_estimators=300,
        max_depth=None,
        min_samples_split=3,
        min_samples_leaf=2,
        max_features="sqrt",
        bootstrap=True,
        n_jobs=-1,
        random_state=42,
    )),
])

et_pipe_orig = Pipeline([
    ("preprocess", preprocessor_orig),
    ("model", ExtraTreesRegressor(
        n_estimators=400,
        max_depth=None,
        min_samples_leaf=2,
        max_features="sqrt",
        bootstrap=False,
        n_jobs=-1,
        random_state=42,
    )),
])

svr_pipe_orig = Pipeline([
    ("preprocess", preprocessor_orig),
    ("model", SVR(
        kernel="rbf",
        C=10,
        epsilon=0.1,
        gamma="scale",
    )),
])

stack_pipe_orig = StackingRegressor(
    estimators=[
        ("elastic_orig", elastic_pipe_orig),
        ("hgb_orig", hgb_pipe_orig),
        ("rf_orig", rf_pipe_orig),
    ],
    final_estimator=HistGradientBoostingRegressor(
        learning_rate=0.05,
        max_depth=5,
        l2_regularization=0.5,
        random_state=42,
    ),
    passthrough=False,
    n_jobs=-1,
)

##### 4.3.2 Using optimized Preprocessing

In [ ]:
def create_model_pipe(prepro_pipe, fs_pipe, model):
    model_pipe = Pipeline([
        ("preprocess", prepro_pipe),
        ("fs", fs_pipe),
        ("model", model),
    ])
    return model_pipe


### LINEAR MODEL
# ElasticNet
elastic_net_model = ElasticNet(
        alpha=0.01,
        l1_ratio=0.5,
        max_iter=30000,
        tol=1e-4,
        selection="cyclic",
        random_state=42,
    )
elastic_pipe_fe = create_model_pipe(preprocessor_pipe, fs_pipe_remove_redundant, elastic_net_model)

### TREE MODELS
# HistGradientBoostingRegressor
hgb_model = HistGradientBoostingRegressor(
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=20,
        l2_regularization=0.5,
        random_state=42,
    )
hgb_pipe_fe = create_model_pipe(preprocessor_pipe, fs_pipe_keep_redundant, hgb_model)

# RandomForestRegressor
rf_model = RandomForestRegressor(
        n_estimators=300,
        max_depth=None,
        min_samples_split=3,
        min_samples_leaf=2,
        max_features="sqrt",
        bootstrap=True,
        n_jobs=-1,
        random_state=42,
    )
rf_pipe_fe = create_model_pipe(preprocessor_pipe, fs_pipe_keep_redundant, rf_model)

# ExtraTreesRegressor
et_model = ExtraTreesRegressor(
        n_estimators=400,
        max_depth=None,
        min_samples_leaf=2,
        max_features="sqrt",
        bootstrap=False,
        n_jobs=-1,
        random_state=42,
    )
et_pipe_fe = create_model_pipe(preprocessor_pipe, fs_pipe_keep_redundant, et_model)

### KERNEL-BASED MODEL (SVR)
svr_model = SVR(
        kernel="rbf",
        C=10,
        epsilon=0.1,
        gamma="scale",
    )
svr_pipe_fe = create_model_pipe(preprocessor_pipe, fs_pipe_remove_redundant, svr_model)

### ENSEMBLE META MODEL (Stacking)
stack_pipe_fe = StackingRegressor(
    estimators=[
        ("hgb_fe", hgb_pipe_fe),
        ("rf_fe", rf_pipe_fe),
        ("et_fe", et_pipe_fe),
    ],
    final_estimator=HistGradientBoostingRegressor(
        learning_rate=0.05,
        max_depth=5,
        l2_regularization=0.5,
        random_state=42,
    ),
    passthrough=False,
    n_jobs=-1,
)

#### 4.4 First run of models

In [ ]:
# # TODO uncomment (currently its commented to save time during experimentation)

# # First evaluation of metrics based on original and engineered feature pipeline to decide how to proceed


# models_orig = {
#     # "ElasticNet_orig": elastic_pipe_orig,
#     "HGB_orig": hgb_pipe_orig,
#     "RF_orig": rf_pipe_orig,
#     "ET_orig": et_pipe_orig,
#     "SVR_orig": svr_pipe_orig,
#     "Stack_orig": stack_pipe_orig,
# }

# models_fe = {
#     # "ElasticNet_fe": elastic_pipe_fe,
#     "HGB_fe": hgb_pipe_fe,
#     "RF_fe": rf_pipe_fe,
#     "ET_fe": et_pipe_fe,
#     "SVR_fe": svr_pipe_fe,
#     "Stack_fe": stack_pipe_fe,
# }

# results = []

# # for name, model in {**models_orig, **models_fe}.items():
#     print(f"Fitting {name} with cross-validation...")
    
#     # Perform cross-validation on the entire training set
#     cv_results = cross_validate(
#         model, 
#         X_train, 
#         y_train,
#         cv=3,
#         scoring={
#             'neg_mae': 'neg_mean_absolute_error',
#             'neg_mse': 'neg_mean_squared_error',
#             'r2': 'r2'
#         },
#         return_train_score=False,
#         verbose=3,
#         n_jobs=-2
#     )
    
#     # Calculate mean metrics across folds
#     mae = -cv_results['test_neg_mae'].mean()
#     rmse = np.sqrt(-cv_results['test_neg_mse'].mean())
#     r2 = cv_results['test_r2'].mean()
    
#     results.append({
#         "model": name,
#         "feature_set": "original" if name.endswith("_orig") else "engineered",
#         "MAE": mae,
#         "RMSE": rmse,
#         "R2": r2,
#     })

# results_df = (
#     pd.DataFrame(results)
#       .sort_values(["feature_set", "MAE"])
#       .reset_index(drop=True)
# )

# print(results_df)

# # Long Duration (with orig ca 25mins VS without orig ca 6mins VS with CV ca 16mins VS with njobs=-1 ca )

# # Predicted on hold-out val set (20%):
# #       model feature_set          MAE          RMSE        R2
# # 0     RF_fe  engineered  1299.728938  4.509435e+06  0.950490
# # 1  Stack_fe  engineered  1321.130612  4.831609e+06  0.946953
# # 2     ET_fe  engineered  1328.051439  4.707534e+06  0.948315
# # 3    HGB_fe  engineered  1534.496164  5.609255e+06  0.938415
# # 4    SVR_fe  engineered  2955.064750  3.242891e+07  0.643956

# # Predicted using 3-fold CV on entire data:
# #       model feature_set          MAE         RMSE        R2
# # 0     RF_fe  engineered  1336.806163  2375.850617  0.940424
# # 1  Stack_fe  engineered  1357.266391  2505.029128  0.933786
# # 2     ET_fe  engineered  1364.212656  2399.654669  0.939223
# # 3    HGB_fe  engineered  1551.419964  2503.445871  0.933858
# # 4    SVR_fe  engineered  3068.524237  6130.420383  0.603579

### 5. Hyperparameter Tuning and Model Evaluation

**Our approach:**
- After first experiments we decided to skip hyperparameter-tuning for SVR and ET

##### 5.1 ElasticNet

In [ ]:
# TODO this cell is commented because we dont evaluate elasticnet for final performance (save time)
# # Hyperparameter Tuning: ElasticNet

# elastic_param_grid = {
#     "model__alpha": [0.001],    # also tried 0.01, 0.05, 0.1, 0.5
#     "model__l1_ratio": [0.9]    # also tried 0.1, 0.3, 0.5, 0.7  
# }

# # CV: Calculate all metrics but use MAE for selecting best model
# elastic_grid = GridSearchCV(
#     elastic_pipe_fe, 
#     param_grid=elastic_param_grid,
#     cv=5,
#     scoring={
#         'mae': 'neg_mean_absolute_error',
#         'mse': 'neg_mean_squared_error',
#         'r2': 'r2'
#     },
#     refit='mae', # Refit the best model based on MAE on the whole training set
#     n_jobs=-2,
#     verbose=3,
#     return_train_score=False
# )
# elastic_grid.fit(X_train, y_train)

# # Get mean metrics across folds
# mae = -elastic_grid.cv_results_['mean_test_mae'][elastic_grid.best_index_]
# mse = -elastic_grid.cv_results_['mean_test_mse'][elastic_grid.best_index_]
# rmse = np.sqrt(mse)
# r2 = elastic_grid.cv_results_['mean_test_r2'][elastic_grid.best_index_]
# print("ElasticNet Results (CV on entire train set):")
# print(f"MAE: {mae:.4f}")
# print(f"RMSE: {rmse:.4f}")
# print(f"R²: {r2:.4f}")
# print("Best ElasticNet params:", elastic_grid.best_params_)

# elastic_best = elastic_grid.best_estimator_ # Final model trained on entire training set with best hyperparameters minimizing MAE

# # Long Duration (Before removal of OHE-categoricals interrupted kernel after 64mins VS after removal ca 1min -> now 15secs with njobs=-2)

# # ElasticNet Results: 
# # MAE: 2353.9112 | RMSE: 13356867.7860 | R2: 0.8534
# # Best ElasticNet params: {'model__alpha': 0.001, 'model__l1_ratio': 0.9}

# # MAE: 2589.6100
# # RMSE: 4104.4515
# # R²: 0.8222
# # Best ElasticNet params: {'model__alpha': 0.001, 'model__l1_ratio': 0.9}

In [ ]:
# # TODO this cell is commented because of time constraints
# # Use GridSearchCV for features_to_select
# # Base model: tuned ElasticNet from above
# en_base = clone(elastic_best.named_steps["model"])

# # Pipeline: clean preprocessing -> RFE -> model
# rfe_pipe_linear = Pipeline([
#     ("preprocess", preprocessor_fe_clean),
#     ("rfe", RFE(
#         estimator=en_base,
#         step=0.5,               # drop ~20% per iteration
#         importance_getter="auto"
#     )),
#     ("model", clone(en_base))
# ])

# # Try a few target feature counts (adjust as needed)
# number_of_all_features = preprocessor_fe_clean.transform(X_train).shape[1]
# rfe_param_grid = {
#     "rfe__n_features_to_select": [int(number_of_all_features*0.5)]# , int(number_of_all_features*1)] # use only these two extremes to save time ~J
# }

# rfe_grid = GridSearchCV(
#     rfe_pipe_linear,
#     param_grid=rfe_param_grid,
#     cv=2,
#     scoring="neg_mean_absolute_error",
#     n_jobs=-1,
#     verbose=3,
#     return_train_score=False,
# )

# rfe_grid.fit(X_train, y_train)

# print("Best n_features_to_select:", rfe_grid.best_params_["rfe__n_features_to_select"])
# print("MAE (CV):", -rfe_grid.best_score_)
# rfe_best = rfe_grid.best_estimator_

# # list kept features
# best_rfe = rfe_best.named_steps["rfe"]
# all_feats = rfe_best.named_steps["preprocess"].get_feature_names_out()
# kept = [f for f, keep in zip(all_feats, best_rfe.support_) if keep]
# print("Kept features:", kept)


**Reasoning**: We used 100 features as an initial, arbitrary cutoff for feature selection in the ElasticNet model. Preliminary experiments and insights from the EDA (see separate notebook) indicated that tree-based methods are likely to perform better. Therefore, we prioritized feature selection for the tree-based models based on SHAP values.
 

##### 5.2 HistGradientBoost

In [ ]:
hgb_param_dist = {
    "fs__vt__threshold": [0.0, 0.005, 0.01],
    "model__learning_rate": uniform(0.01, 0.15),       # samples values
    "model__max_leaf_nodes": randint(50, 170),         
    "model__min_samples_leaf": randint(2, 20),         # samples leaf sizes between 2–20
    "model__max_iter": randint(200, 900),              # tries 200–900 iterations
    "model__l2_regularization": uniform(0.0, 1.0),      # samples small regularization values
    "model__early_stopping": [True],
    "model__validation_fraction": [0.1],
    "model__n_iter_no_change": [20],
    "model__random_state":[42]
}

# optimized the parameter distributions based on previous runs to focus search space
hgb_param_dist = {
    "fs__vt__threshold": [0.0],
    "model__learning_rate": [0.05889383578028271],
    "model__max_leaf_nodes": [139],
    "model__min_samples_leaf": [4],
    "model__max_iter": [602],
    "model__l2_regularization": [0.8583588048137198],
    "model__early_stopping": [True],
    "model__validation_fraction": [0.1],
    "model__n_iter_no_change": [20],
    "model__random_state":[42]
}
    
hgb_best = model_hyperparameter_tuning(X_train, y_train, hgb_pipe_fe, hgb_param_dist, n_iter=100, splits=5)

# Before FS (last step was: Add "mpg_x_age", "tax_x_age")
# MAE: 1335.6473
# RMSE: 2308.0557
# R²: 0.9439
# Best Model params: {'model__validation_fraction': 0.1, 'model__random_state': 42, 'model__n_iter_no_change': 20, 'model__min_samples_leaf': 4, 'model__max_leaf_nodes': 139, 'model__max_iter': 602, 'model__learning_rate': 0.05889383578028271, 'model__l2_regularization': 0.8583588048137198, 'model__early_stopping': True, 'fs__vt__threshold': 0.0}

# use mean instead of median in  "age_rel_brand" because most of the values were 0 otherwise
# MAE: 1323.1182
# RMSE: 2300.0353
# R²: 0.9443
# Best Model params: {'model__validation_fraction': 0.1, 'model__random_state': 42, 'model__n_iter_no_change': 20, 'model__min_samples_leaf': 4, 'model__max_leaf_nodes': 139, 'model__max_iter': 602, 'model__learning_rate': 0.05889383578028271, 'model__l2_regularization': 0.8583588048137198, 'model__early_stopping': True, 'fs__vt__threshold': 0.0}

##### 5.3 RandomForest

In [ ]:
# Old parameter distribution
rf_param_dist = {
    "fs__vt__threshold": [0.0, 0.005, 0.01],
    "model__n_estimators": randint(200, 600),        # number of trees
    "model__max_depth": randint(5, 40),              # depth of each tree
    "model__min_samples_split": randint(2, 10),      # min samples to split an internal node
    "model__min_samples_leaf": randint(1, 8),        # min samples per leaf
    "model__max_features": ["sqrt"],           # feature sampling strategy (sqrt performed better than log2 and None in previous tests)
    "model__bootstrap": [False]                      # use bootstrapping or not (False performed better than True in previous tests)
}

# So far best parameter distribution based on previous runs to focus search space
rf_param_dist = {
    "fs__vt__threshold": [0.005],
    "model__n_estimators": [328],
    "model__max_depth": [20],
    "model__min_samples_split": [5],
    "model__min_samples_leaf": [1],
    "model__max_features": ["sqrt"],
    "model__bootstrap": [False],
}

rf_best_rand = model_hyperparameter_tuning(X_train, y_train, rf_pipe_fe, rf_param_dist)

joblib.dump(rf_best_rand, "rf_best_rand.pkl")


# Long Duration (~1min)

# Before FS (last step was: Add "mpg_x_age", "tax_x_age")
# MAE: 1310.7979
# RMSE: 2313.0862
# R²: 0.9437
# Best Model params: {'model__n_estimators': 328, 'model__min_samples_split': 5, 'model__min_samples_leaf': 1, 'model__max_features': 'sqrt', 'model__max_depth': 20, 'model__bootstrap': False, 'fs__vt__threshold': 0.005}

# use mean instead of median in  "age_rel_brand" because most of the values were 0 otherwise
# MAE: 1309.6709
# RMSE: 2309.8228
# R²: 0.9438
# Best Model params: {'model__n_estimators': 328, 'model__min_samples_split': 5, 'model__min_samples_leaf': 1, 'model__max_features': 'sqrt', 'model__max_depth': 20, 'model__bootstrap': False, 'fs__vt__threshold': 0.005}

In [ ]:
pipe = rf_best_rand if hasattr(rf_best_rand, "named_steps") else rf_best_rand[0]
pre = pipe.named_steps["preprocess"]
ct = pre.named_steps["ct"]
feat_names_before_fs = get_feature_names_from_preprocessor(ct) # names after preprocessing

# Get mask from feature selection step to see which features were kept
vt_step = pipe.named_steps["fs"].named_steps["vt"]
mask = vt_step.get_support()

# Apply mask to get final feature names and get feature importances
feat_names = np.array(feat_names_before_fs)[mask]
importances = pipe.named_steps["model"].feature_importances_

feature_importance_df = pd.DataFrame(
    {"feature": feat_names, "importance": importances}
).sort_values("importance", ascending=False)

print("Feature Importances:")
for _, row in feature_importance_df.iterrows():
    print(f"{row['feature']:30s}: {row['importance']:.6f}")

In [ ]:
stop here

##### 5.4 StackingRegressor

In [ ]:
# Old parameter distribution
stack_param_dist = {
    "final_estimator__learning_rate": uniform(0.02, 0.1),
    "final_estimator__max_depth": randint(3, 10),
    "final_estimator__min_samples_leaf": randint(3, 20),
    "final_estimator__l2_regularization": uniform(0.0, 1.0),
}

# So far best parameter distribution based on previous runs to focus search space
stack_param_dist = {
    "final_estimator__learning_rate": [0.061135390505667866],
    "final_estimator__max_depth": [5],
    "final_estimator__min_samples_leaf": [10],
    "final_estimator__l2_regularization": [0.19438003399487302]
}

stack_best = model_hyperparameter_tuning(X_train, y_train, stack_pipe_fe, stack_param_dist, splits=3)
# joblib.dump(stack_best, "stack_best.pkl")


# Long Duration (~3mins)

# MAE: 1351.8682
# RMSE: 2498.2822
# R²: 0.9342

# After RandomizedSearchCV:
# MAE: 1350.4717
# RMSE: 2497.0474
# R²: 0.9343
# Best Model params: {'final_estimator__l2_regularization': np.float64(0.978892858275009), 'final_estimator__learning_rate': np.float64(0.06867421529594551), 'final_estimator__max_depth': 6, 'final_estimator__min_samples_leaf': 13}

# Removed ElasticNet from stacking due to poor performance compared to RF and HGB alone
# canceled but the cv scores didnt seem to show much improvement

# Using transmission and fuelType as OHE instead of TE():
# MAE: 1357.4291
# RMSE: 2516.5470
# R²: 0.9333
# Best Model params: {'final_estimator__l2_regularization': np.float64(0.19438003399487302), 'final_estimator__learning_rate': np.float64(0.061135390505667866), 'final_estimator__max_depth': 5, 'final_estimator__min_samples_leaf': 10}


# Removed fillna(0) in feature engineering for a_x_b and model_freq():
# was worse for hgb and rf so not tested for stacking

# ...

# implemented GroupModeImputer
# MAE: 1329.2379
# RMSE: 2453.0239
# R²: 0.9366
# Best Model params: {'final_estimator__min_samples_leaf': 10, 'final_estimator__max_depth': 5, 'final_estimator__learning_rate': 0.061135390505667866, 'final_estimator__l2_regularization': 0.19438003399487302}

# Fixed GroupImputer and added Feature Engineering to pipeline
# MAE: 1369.6876
# RMSE: 2516.2583
# R²: 0.9333


### 6. Feature Importance of Tree Models (with SHAP)

Use SHAP (SHapley Additive exPlanations) to
  identify feature importance specifically for our selected tree models (HGB&RF)

  **Why SHAP for Trees:**
  - Provides exact feature importance values for tree-based
  models
  - Tree models handle irrelevant features, but noise features
  still impact performance
  - Enables data-driven selection rather than statistical filter
  methods

#### 6.1 Define needed Functions

In [ ]:
# Get Feature names aligned with X_proc
def get_pipeline_feature_matrix(pipe, X):
    """
    Given a fitted pipeline with steps:
      'preprocess' -> optional 'vt' -> optional 'fs' -> 'model'
    return:
      X_proc: 2D numpy array of features just before the model step
      feat_names: 1D np.array of feature names aligned with X_proc columns
    """
    # 1) Preprocess
    pre = pipe.named_steps["preprocess"]
    X_proc = pre.transform(X)

    # --- get feature names from the ColumnTransformer inside 'pre' ---
    # pre is a Pipeline: ('fe', CarFeatureEngineer) -> ('group_imputer', GroupImputer) -> ('ct', ColumnTransformer)
    if isinstance(pre, Pipeline) and "ct" in pre.named_steps:
        ct = pre.named_steps["ct"]
        feat_names = np.asarray(get_feature_names_from_preprocessor(ct), dtype=object)
    elif hasattr(pre, "get_feature_names_out"):
        feat_names = np.asarray(pre.get_feature_names_out(), dtype=object)
    else:
        # fallback: generic names
        feat_names = np.asarray([f"f_{i}" for i in range(X_proc.shape[1])], dtype=object)

    # 2) VarianceThreshold (optional)
    vt = pipe.named_steps.get("vt")
    if vt is not None:
        mask_vt = vt.get_support()
        X_proc = vt.transform(X_proc)
        feat_names = feat_names[mask_vt]

    # 3) SelectFromModel (optional FS step)
    fs = pipe.named_steps.get("fs")
    if fs is not None:
        mask_fs = fs.get_support()
        X_proc = fs.transform(X_proc)
        feat_names = feat_names[mask_fs]

    return X_proc, feat_names


In [ ]:
# Compute Shap Importance
def compute_shap_importance(
    pipe,
    X,
    sample_size=1000,
    seed=42,
    model_name=None,
):
    """
    Compute global SHAP feature importances for a fitted pipeline.

    Steps:
      - Transform X with the pipeline up to just before the model.
      - Subsample up to `sample_size` rows.
      - Use TreeExplainer on the model (tree-based models).
      - Return a DataFrame with mean |SHAP| per feature.

    Returns:
      shap_df: DataFrame with columns ['feature', 'importance']
      feat_names: np.array of feature names aligned with importances
    """
    # Extract processed feature matrix and names
    X_proc, feat_names = get_pipeline_feature_matrix(pipe, X)

    # Subsample rows for SHAP (for speed)
    rng = np.random.default_rng(seed)
    n = min(sample_size, len(X_proc))
    idx = rng.choice(len(X_proc), n, replace=False)
    X_sample = X_proc[idx]

    # Underlying model (last step in pipeline)
    model = pipe.named_steps["model"]
    tag = model_name or model.__class__.__name__

    # TreeExplainer is appropriate for tree ensembles (RF, ET, HGB, GB, etc.)
    explainer = shap.TreeExplainer(model)
    shap_vals = explainer.shap_values(X_sample)

    # For regression, shap_vals is (n_samples, n_features)
    importance = np.abs(shap_vals).mean(axis=0)

    shap_df = (
        pd.DataFrame({"feature": feat_names, "importance": importance})
        .sort_values("importance", ascending=False)
        .reset_index(drop=True)
    )

    print(f"Top 20 features by SHAP for {tag}:")
    print(shap_df.head(20).to_string(index=False))

    return shap_df, feat_names


In [ ]:
# Plot Shap Importance
def plot_top_shap(shap_df, model_name, top_k=20):
    """
    Horizontal bar plot of top_k features by mean |SHAP|.
    """
    top_df = shap_df.head(top_k).iloc[::-1]  # reverse for nicer barh order

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.barh(top_df["feature"], top_df["importance"])
    ax.set_xlabel("Average |SHAP| value")
    ax.set_title(f"Top {top_k} features by SHAP – {model_name}")
    plt.tight_layout()
    plt.show()


In [ ]:
# See if smaller k features improve MAE (top k)
def cv_mae_topk_from_shap(
    pipe,
    shap_importance,
    X,
    y,
    n_features_list,
    folds=5,
    seed=42,
    model_name=None,
):
    """
    For a fitted pipeline `pipe` and its SHAP importances:
      - Build X_proc, feat_names from the pipeline.
      - For each k in n_features_list:
          * Take top-k features by SHAP.
          * Run KFold CV on X_proc[:, idx] with the pipeline's final estimator.
      - Print MAE per k and return the best (k, model, feature list).

    Returns:
      best_model: fitted estimator on full X_proc restricted to best-k features
      best_features: list of feature names used
    """
    # 1) Get processed features and names
    X_proc, feat_names = get_pipeline_feature_matrix(pipe, X)
    feat_names = np.asarray(feat_names, dtype=object)

    # 2) SHAP ranking
    shap_sorted = shap_importance.sort_values("importance", ascending=False)
    shap_order = shap_sorted["feature"].tolist()

    # helper: indices of top-k by SHAP
    def indices_for_topk(k):
        top_feats = shap_order[:k]
        return [i for i, fname in enumerate(feat_names) if fname in top_feats]

    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    model_proto = pipe.named_steps["model"]
    tag = model_name or model_proto.__class__.__name__

    results = []

    for k in n_features_list:
        idx = indices_for_topk(k)
        if len(idx) == 0:
            print(f"Skipping k={k}: no matching feature indices.")
            continue

        mae_folds = []

        for train_idx, val_idx in kf.split(X_proc):
            X_tr, X_val = X_proc[train_idx][:, idx], X_proc[val_idx][:, idx]
            y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

            est = clone(model_proto)
            est.fit(X_tr, y_tr)
            y_pred = est.predict(X_val)
            mae_folds.append(mean_absolute_error(y_val, y_pred))

        mae_mean = float(np.mean(mae_folds))
        results.append({"k": k, "mae": mae_mean, "idx": idx})

    # pick best k
    if not results:
        raise RuntimeError("No valid k in n_features_list produced results.")

    best = min(results, key=lambda r: r["mae"])
    best_k = best["k"]
    best_mae = best["mae"]
    best_idx = best["idx"]
    best_features = [feat_names[i] for i in best_idx]

    print(f"\nTop-k SHAP feature CV – {tag}")
    for r in results:
        print(f"  k={r['k']:3d} | MAE={r['mae']:.2f}")
    print(f"Best: k={best_k} | MAE={best_mae:.2f}")

    # fit final estimator on full X_proc restricted to best_k features
    final_est = clone(model_proto)
    final_est.fit(X_proc[:, best_idx], y)

    return final_est, best_features


In [ ]:
class ShapTopKColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer that selects a fixed subset of columns by name.

    Parameters
    ----------
    selected_features : list of str
        Feature names (after preprocessing) to keep.

    all_feature_names : array-like of str
        Full list of feature names aligned with the columns of X after preprocessing.
        These are typically obtained from get_pipeline_feature_matrix(...).
    """
    def __init__(self, selected_features, all_feature_names):
        self.selected_features = list(selected_features)
        self.all_feature_names = np.asarray(all_feature_names, dtype=object)

    def fit(self, X, y=None):
        # Compute the column indices corresponding to selected_features
        name_to_idx = {name: i for i, name in enumerate(self.all_feature_names)}
        self.idx_ = [
            name_to_idx[name]
            for name in self.selected_features
            if name in name_to_idx
        ]
        if len(self.idx_) == 0:
            raise ValueError(
                "ShapTopKColumnSelector: none of the selected_features were found "
                "in all_feature_names."
            )
        return self

    def transform(self, X):
        # X is the matrix after preprocessing; select only the chosen columns
        return X[:, self.idx_]

    def get_feature_names_out(self, input_features=None):
        # For consistency with sklearn's feature-name API
        return np.asarray(self.selected_features, dtype=object)


In [ ]:
def build_shap_topk_pipeline(
    base_pipe,
    best_features,
    all_feature_names,
    step_model_name="model",
):
    """
    Build a final pipeline that:
      - reuses the preprocessing (and vt/fs if present) from `base_pipe`
      - inserts a ShapTopKColumnSelector to keep only `best_features`
      - uses a fresh clone of the base model as final estimator

    Parameters
    ----------
    base_pipe : sklearn Pipeline
        Fitted pipeline with steps: 'preprocess' -> optional 'vt'/'fs' -> 'model'.

    best_features : list of str
        Names of the features (after preprocessing) to keep.

    all_feature_names : array-like of str
        Full list of feature names aligned with the output of preprocessing
        (and vt/fs if they were applied when computing SHAP).

    step_model_name : str, default="model"
        Name of the final estimator step in base_pipe.

    Returns
    -------
    final_pipe : sklearn Pipeline
        Unfitted pipeline. Call final_pipe.fit(X, y) to train on full data.
    """
    steps = []

    # 1) Preprocess step (clone so we refit on full data)
    pre = base_pipe.named_steps["preprocess"]
    steps.append(("preprocess", clone(pre)))

    # 2) Optional VarianceThreshold
    if "vt" in base_pipe.named_steps and base_pipe.named_steps["vt"] is not None:
        steps.append(("vt", clone(base_pipe.named_steps["vt"])))

    # 3) Optional SelectFromModel (fs) – only if you actually use it
    if "fs" in base_pipe.named_steps and base_pipe.named_steps["fs"] is not None:
        steps.append(("fs", clone(base_pipe.named_steps["fs"])))

    # 4) SHAP-based column selector
    shap_selector = ShapTopKColumnSelector(
        selected_features=best_features,
        all_feature_names=all_feature_names,
    )
    steps.append(("shap_select", shap_selector))

    # 5) Final estimator – fresh clone of the base model
    base_model = base_pipe.named_steps[step_model_name]
    steps.append(("model", clone(base_model)))

    final_pipe = Pipeline(steps)
    return final_pipe


#### 6.2 Feature Importance HGB

In [ ]:
# Unpack tuned pipeline and search object
hgb_pipe, hgb_search = hgb_best

# Best CV scores from the search
idx = hgb_search.best_index_
mae_cv  = -hgb_search.cv_results_['mean_test_mae'][idx]
rmse_cv = np.sqrt(-hgb_search.cv_results_['mean_test_mse'][idx])
r2_cv   = hgb_search.cv_results_['mean_test_r2'][idx]

# Feature count after preprocess (+ VT)
X_proc_hgb, feat_names_hgb = get_pipeline_feature_matrix(hgb_pipe, X_train)
n_features_total_hgb = X_proc_hgb.shape[1]

print("Baseline HGB (CV on train):")
print(f"MAE:  {mae_cv:.4f}")
print(f"RMSE: {rmse_cv:.4f}")
print(f"R²:   {r2_cv:.4f}")
print(f"Total features used: {n_features_total_hgb}")


In [ ]:
shap_importance_hgb, feat_names_hgb = compute_shap_importance(
    hgb_pipe,
    X_train,
    sample_size=1000,
    seed=42,
    model_name="HGB",
)

plot_top_shap(shap_importance_hgb, model_name="HGB", top_k=n_features_total_hgb)


In [ ]:
# Check if a smaller feature amount gives better MAE
n_features_list_hgb = list(range(10, n_features_total_hgb)) 
if n_features_total_hgb not in n_features_list_hgb:
    n_features_list_hgb.append(n_features_total_hgb)

best_model_hgb, best_features_hgb = cv_mae_topk_from_shap(
    pipe=hgb_pipe,
    shap_importance=shap_importance_hgb,
    X=X_train,
    y=y_train,
    n_features_list=n_features_list_hgb,
    folds=5,
    seed=42,
    model_name="HGB",
)


In [ ]:
# Final Pipe with best k and MAE
hgb_final_shap_pipe = build_shap_topk_pipeline(
    base_pipe=hgb_pipe,
    best_features=best_features_hgb,
    all_feature_names=feat_names_hgb,
    step_model_name="model",   # name of the final estimator step in hgb_pipe
)

# Fit on full training data
hgb_final_shap_pipe.fit(X_train, y_train)

# Save for later use
joblib.dump(hgb_final_shap_pipe, "hgb_final_shap_pipe.pkl")


#### 6.3 Feature Importance RF

In [ ]:
# Unpack tuned RF pipeline and search object
rf_pipe, rf_search = rf_best_rand  # result from model_hyperparameter_tuning

# Best CV scores from the search
idx = rf_search.best_index_
mae_cv_rf  = -rf_search.cv_results_['mean_test_mae'][idx]
rmse_cv_rf = np.sqrt(-rf_search.cv_results_['mean_test_mse'][idx])
r2_cv_rf   =  rf_search.cv_results_['mean_test_r2'][idx]

# Feature matrix + names after preprocess (+ vt/fs if present)
X_proc_rf, feat_names_rf = get_pipeline_feature_matrix(rf_pipe, X_train)
n_features_total_rf = X_proc_rf.shape[1]

print("Baseline RandomForest (CV on train):")
print(f"MAE:  {mae_cv_rf:.4f}")
print(f"RMSE: {rmse_cv_rf:.4f}")
print(f"R²:   {r2_cv_rf:.4f}")
print(f"Total features used: {n_features_total_rf}")

In [ ]:
shap_importance_rf, feat_names_rf_check = compute_shap_importance(
    rf_pipe,
    X_train,
    sample_size=1000,
    seed=42,
    model_name="RandomForest",
)

plot_top_shap(shap_importance_rf, model_name="RandomForest", top_k=n_features_total_rf)


In [ ]:
# Check if a smaller feature amount gives better MAE
n_features_list_rf = list(range(10, n_features_total_rf)) 
if n_features_total_rf not in n_features_list_rf:
    n_features_list_rf.append(n_features_total_rf)

best_model_rf, best_features_rf = cv_mae_topk_from_shap(
    pipe=rf_pipe,
    shap_importance=shap_importance_rf,
    X=X_train,
    y=y_train,
    n_features_list=n_features_list_rf,
    folds=5,
    seed=42,
    model_name="RandomForest",
)

In [ ]:
# Build final pipeline:
#   preprocess -> (vt/fs) -> shap_select(best_features_rf) -> RF
rf_final_shap_pipe = build_shap_topk_pipeline(
    base_pipe=rf_pipe,
    best_features=best_features_rf,
    all_feature_names=feat_names_rf,
    step_model_name="model",   # name of the RF step in rf_pipe
)

# Fit final RF SHAP-top-k pipeline on full training data
rf_final_shap_pipe.fit(X_train, y_train)

# Optionally save for later use
joblib.dump(rf_final_shap_pipe, "rf_final_shap_pipe.pkl")

### 7. Kaggle Competition

Extra Task (1 Point): Be in the Top 5 Groups on Kaggle

In [ ]:
def predict_on_test(model_pipeline, model_name):
    # Load best model from Joblib and predict on validation set to verify
    pipe_best = joblib.load(model_pipeline)
    
    # Predict on test set
    df_cars_test['price'] = pipe_best.predict(df_cars_test)
    df_cars_test['price'].to_csv(f'Group05_{model_name}_Version10.csv', index=True)

In [ ]:
predict_on_test("hgb_final_shap_pipe.pkl", "HGB")

In [ ]:
predict_on_test("rf_final_shap_pipe.pkl", "RF")

In [ ]:
# predict_on_test("stack_pipe.pkl", "Stack")

In [ ]:
# !kaggle competitions submit -c cars4you -f Group05_Version05.csv -m "Message" # Uncomment to submit to Kaggle

In [ ]:
!kaggle competitions submissions -c cars4you

### 8. Open-Ended-Section

#### 8.1 Counterfactual Sensitivity Maps

**a) Objective and motivation (0.5v)**

We have a trained model that predicts a car's price from features (age, mileage, engine size, paint quality, mpg, etc.). For a single car, we want to answer:

_“What small changes to the car's features would be required to make the model predict a x% higher or lower price?”_ (we tried +-10)

That is a counterfactual question: “If this car were different in X ways, would the model value it differently?”

Instead of only knowing which features are important overall, this tells  for this exact car what the model would need to see to change its valuation.

Examples of use:
- If the counterfactual says “increase paintQuality by 5 will increase the price by +10%”, a business could decide whether doing that repair is worth it.
- If the model requires impossible changes to reach +10%, the car should be flagged for manual inspection or considered non-upgradeable.

------------
**Limitations:**
- The greedy search is heuristic — it is fast and interpretable but not guaranteed to find the global minimal change. It is designed for practical interpretability rather than mathematical optimality.
- Categorical changes (e.g., changing `Brand` or `model`) are **not** attempted by default to avoid generating invalid combinations; can be added later if you provide a realistic whitelist of allowed swaps.
- Some input features are physically immutable or economically infeasible to change; results must be interpreted by a domain expert.
-----


**b) Difficulty of tasks (1v)**

Implementing counterfactual sensitivity analysis in this project is non-trivial because:

1. **Complex Preprocessing Pipeline:** The model uses hierarchical imputers, log transforms, target encodings, scaling, and many engineered features.
All counterfactuals must operate in the raw input space, while still being compatible with the full sklearn pipeline.

2. **Non-Differentiable Model:** Our final model is tree-based and does not provide gradients. Therefore, we implemented a greedy local search algorithm: Perturb the most influential numeric features (ranked via permutation importance).
Accept only changes that monotonically move the prediction closer to the target.
Shrink step sizes adaptively when progress stalls.
Enforce realistic value ranges based on training quantiles.

3. **Handling Mixed Pandas Dtypes:** The real dataset uses Int64 nullable types, which break when assigning floats. All numeric fields were therefore converted to consistent float64 inputs before search.

-----

**c) Correctness and efficiency of implementation (1v)**

We used a greedy, local search that operates in the input (raw) feature space. Our idea:

1. Pick one car (one row of features).
2. Compute the current predicted price using the full pipeline.
3. Decide a target: e.g., +10% (increase) or -10% (decrease).
4. Pick a short list of numeric features that the model tends to rely on (we rank features by permutation importance on a training sample).
5. Iteratively nudge each of those features a little bit in the direction that would move the prediction toward the target: 
    - Step sizes are based on realistic feature variation from the training set (q10→q90 range).
    - After each proposed change we run the entire pipeline and get a new prediction.
    - We keep the change only if it moved the prediction closer to the target.
    - If nothing improves, we reduce step sizes and try again, for up to a maximum number of iterations.
6. Stop when the target is reached or we run out of iterations / step size becomes tiny.

Important implementation details to make this robust:
- Coerce all numeric inputs to float (to avoid pandas nullable-int assignment errors).
- Fill missing numeric inputs with training medians before searching so the pipeline will not break.
- Bound proposed values within training min/max to avoid unrealistic values.
- Use a deterministic search (no random elements) so results are reproducible.

------

**d) Discussion of results (1v)**

For each sampled test car (we sampled 30):
- orig_pred: the original model prediction for that car.
- target (implicit): the target price we tried to reach (orig_pred x 1.10 for +10%).
- final_pred: the model's prediction after applying the accepted changes.
- success: True if the greedy search reached (or exceeded) the target, False otherwise.
- iters: how many iterations the search ran.
- changes: dictionary of features the search changed and the new values it set for them.

We also saved open_ended_counterfactual_summary.csv with these results.

1. **Success Rates:** +10% target: 6.67%  // -10% target: 0.00%
    - For the sampled cars, only ~1 in 15 (6.7%) could reach a +10% increase by changing realistic features; none could be reduced by 10% using realistic changes.
    - That means most cars are already at the top (or bottom) of what the model considers plausible given their brand, age, engine size, etc. Small **realistic** tweaks cannot move the valuation.

2. **Most used Features** by the Algorithm: miles_per_year: 37 times; mpg: 31 times; tax: 28 times
    - When the algorithm tries to change the prediction, it almost always does so by adjusting these three features. These are the model's “go-to” levers to move predictions locally.

3. **Average Size of Suggested Changes**: mpg: +59.4; miles_per_year: +7,698; tax: +119.6
    - The model requested large changes: +59 mpg or +7.7k miles/year or +120 tax units. These are not small, realistic fixes—large or impossible in practice.
    - Increasing miles per year is not only impossible, it is also unrealistic that more miles would increase the car price. This shows us that the model learnt an unrealistic relationship.
    - This tells the business: even where changes are suggested, they are generally unrealistic, so the car's price is effectively locked by stronger features (brand, age, engine size).

4. **Successful Example** (index 100785): orig_pred = 10,783; final_pred = 12,014 (success True); changes: {'mpg': 54.7, 'tax': 164.5}  (1 iteration)
    - The model was able to push the price up by 10% in one step by dramatically increasing mpg and tax. But those required changes are huge and not realistic — this was a “success” only in the model's internal logic, not a practical recommendation.

5. **Failed example** (index 83321): orig_pred = 14,470; final_pred = 14,746  (success False); changes: {'mpg': 62.35, 'miles_per_year': 6,347.74}  (26 iterations)
    - Even after many iterations and very large changes, the model could not reach a +10% increase. This shows the model is constrained by other features (age/brand/engine) that we did not change and cannot realistically change.

----    

**e) Alignment with objectives (0.5v)**

1. **Clear objective**: Our goal was simple: check what small, realistic changes to a car's features would be needed to move its predicted price by ±10%. This gives practical “what-if” insights instead of only global feature importance.
2. **Non-trivial task**: Although the question is simple, solving it was technically difficult because our model uses a complex preprocessing pipeline and non-linear algorithms. We had to design a custom search method that safely modifies input features and runs the entire pipeline each time.
3. **Correct and efficient implementation**: The solution only changes features that make sense, keeps values within realistic ranges, and automatically handles pipeline steps like scaling and imputing. It is deterministic, fast, and works directly with the real production pipeline.
4. **Meaningful findings**: The results showed: Most cars cannot be pushed ±10% with realistic feature changes. The model often requested unrealistic adjustments (e.g., huge mpg changes), revealing rigid pricing behaviour.
Some features the model tried to use (like increasing miles_per_year) were unrealistic, helping us spot places where the model relies on odd patterns.
5. **Objective achieved**: We obtained exactly what we wanted: clear, car-specific explanations and simple recommendations (“small repairs” vs. “manual review”).
The insights directly support the project's aim of understanding and improving price evaluations.


In [ ]:
# Basic asserts so failures are explicit 
assert "pipe" in globals(), "pipeline object `pipe` not found - set `pipe` to your final Pipeline"
assert "X_train" in globals(), "X_train not found"
assert "y_train" in globals(), "y_train not found"
assert "df_cars_test" in globals(), "df_cars_test not found"

# Build the list of numeric features we will consider
num_candidates = []
if "numeric_features_clean" in globals():
    num_candidates += list(numeric_features_clean)
if "log_features_clean" in globals():
    num_candidates += list(log_features_clean)
if not num_candidates:
    if "numeric_features" in globals():
        num_candidates += list(numeric_features)
    if "numeric_features_for_log" in globals():
        num_candidates += list(numeric_features_for_log)

# keep only columns that actually exist in X_train (avoid KeyErrors)
num_features = [f for f in dict.fromkeys(num_candidates) if f in X_train.columns]
print(f"Numeric features detected ({len(num_features)}): {num_features}")

# Build robust summary statistics (numpy floats only) used for bounding and step sizes.
train_stats = {}
for c in num_features:
    arr = pd.to_numeric(X_train[c], errors="coerce").dropna().to_numpy(dtype="float64")
    if arr.size > 0:
        train_stats[c] = {
            "min": float(np.nanmin(arr)),
            "max": float(np.nanmax(arr)),
            "q10": float(np.nanquantile(arr, 0.10)),
            "q90": float(np.nanquantile(arr, 0.90)),
            "median": float(np.nanmedian(arr)),
            "mean": float(np.nanmean(arr)),
            "std": float(np.nanstd(arr, ddof=1) if arr.size > 1 else 0.0)
        }
    else:
        train_stats[c] = {"min": 0.0, "max": 0.0, "q10": 0.0, "q90": 0.0, "median": 0.0, "mean": 0.0, "std": 0.0}

# quick sanity output for first features
print("Example train stats (first 5 features):")
for i, (k, v) in enumerate(train_stats.items()):
    print(f"  {k}: q10={v['q10']}, median={v['median']}, q90={v['q90']}")
    if i >= 4:
        break


In [ ]:
# Permutation importance gives a robust ranking even with encodings and OHE.

COMPUTE_PERM = True  # flip to False if you want to skip this step and just use the numeric_features ordering

perm_imp = None
if COMPUTE_PERM:
    sample_size = min(2000, len(X_train))
    X_sample = X_train.sample(sample_size, random_state=0)
    y_sample = y_train.loc[X_sample.index]
    print("Computing permutation importance and sampling training data")
    res = permutation_importance(pipe, X_sample, y_sample, n_repeats=6, random_state=42, n_jobs=1)
    # res.importances_mean corresponds to the columns in X_sample
    perm_imp = pd.Series(res.importances_mean, index=X_sample.columns).sort_values(ascending=False)
    # keep only numeric features for ranking
    perm_imp = perm_imp.loc[[c for c in perm_imp.index if c in num_features]]
    print("Top numeric features by permutation importance:\n", perm_imp.head(10))
else:
    print("Permutation importance disabled. Ranked features will default to numeric feature order.")

In [ ]:
# Robust greedy counterfactual search
def find_counterfactual_greedy(
    instance: pd.DataFrame,
    pipe,
    X_train: pd.DataFrame,
    target_rel: float = 0.10,
    mode: str = "realistic",
    step_frac: float = 0.15,
    max_iters: int = 200,
    verbose: bool = False,
    perm_importance_series: pd.Series = None
):
    """
    Greedy, deterministic counterfactual search for a single-row `instance`.
    Returns a dict with original/final predictions, success flag, changed features, and trajectory.

    Key design decisions:
    - Work in input-space only (do not modify internals).
    - Coerce numeric inputs to float64, fill missing with train median.
    - Use training q10-q90 ranges to compute sensible step sizes.
    - 'realistic' mode excludes non-actionable features (e.g. miles_per_year).
    """
    assert instance.shape[0] == 1, "instance must be single-row DataFrame"

    # copy and coerce numeric columns to float64 (fallback to training median)
    inst = instance.copy()
    numeric_present = [f for f in num_features if f in inst.columns]
    for f in numeric_present:
        v = pd.to_numeric(inst.at[inst.index[0], f], errors="coerce")
        if pd.isna(v):
            inst.at[inst.index[0], f] = float(train_stats.get(f, {}).get("median", 0.0))
        else:
            inst.at[inst.index[0], f] = float(v)
    if numeric_present:
        inst[numeric_present] = inst[numeric_present].astype("float64")

    cur = inst.copy()
    original_pred = float(pipe.predict(cur)[0])
    target = original_pred * (1.0 + target_rel)
    direction = 1 if target_rel > 0 else -1

    # realistic allowed features: keep only features that can be reasonably changed
    # NOTE: miles_per_year is excluded because it's not actionable in reality.
    if mode == "realistic":
        allowed = [f for f in ["paintQuality", "tax", "mpg"] if f in num_features]
        # allowed = [f for f in ["paintQuality"] if f in num_features]
    else:
        allowed = [f for f in num_features if f in cur.columns]

    # rank allowed features by permutation importance if available
    if perm_importance_series is not None:
        ranked = [f for f in perm_importance_series.index if f in allowed]
    else:
        ranked = allowed.copy()

    # fallback
    if not ranked:
        ranked = [f for f in num_features if f in cur.columns]

    # compute step sizes from q10->q90
    steps = {}
    for f in ranked:
        s = train_stats.get(f)
        range_q = max(1e-8, (s["q90"] - s["q10"]) if s else step_frac)
        steps[f] = range_q * step_frac

    cur_pred = original_pred
    trajectory = [cur_pred]
    changes = {}
    feat_steps = {f: 0.0 for f in ranked}

    # quick exit if already at target
    if direction * (cur_pred - target) >= 0:
        return {'index': cur.index[0], 'original_pred': original_pred, 'target': target,
                'final_pred': cur_pred, 'success': True, 'iters': 0, 'changes': {}, 'trajectory': trajectory}

    it = 0
    while it < max_iters:
        progressed = False
        for f in ranked:
            old_raw = pd.to_numeric(cur.at[cur.index[0], f], errors="coerce")
            old_val = float(old_raw) if not pd.isna(old_raw) else float(train_stats.get(f, {}).get("median", 0.0))
            step = steps.get(f, 0.0)
            if abs(step) < 1e-12:
                continue

            proposed = old_val + direction * step
            bounds = train_stats.get(f, {"min": -1e12, "max": 1e12})
            proposed = max(float(bounds["min"]), min(float(bounds["max"]), float(proposed)))
            cur.at[cur.index[0], f] = float(proposed)

            try:
                new_pred = float(pipe.predict(cur)[0])
            except Exception:
                # revert and reduce step on failure
                cur.at[cur.index[0], f] = float(old_val)
                steps[f] *= 0.5
                continue

            # accept only if it moves towards the target
            if direction * (new_pred - cur_pred) > 1e-8:
                changes[f] = float(proposed)
                feat_steps[f] += direction * float(step)
                cur_pred = new_pred
                trajectory.append(cur_pred)
                progressed = True
                if verbose:
                    print(f"it {it} | {f}: {old_val:.4f} -> {proposed:.4f} | pred {cur_pred:.2f}")
                if direction * (cur_pred - target) >= 0:
                    break
            else:
                # revert and shrink this feature's step
                cur.at[cur.index[0], f] = float(old_val)
                steps[f] *= 0.5

        if not progressed:
            # shrink all steps if no progress this iteration
            for k in steps:
                steps[k] *= 0.5

        if direction * (cur_pred - target) >= 0:
            break
        if all(abs(s) < 1e-12 for s in steps.values()):
            break
        it += 1

    success = (direction * (cur_pred - target) >= 0)
    return {
        'index': cur.index[0],
        'original_pred': original_pred,
        'target': target,
        'final_pred': cur_pred,
        'success': bool(success),
        'iters': it,
        'changes': changes,
        'trajectory': trajectory,
        'feature_steps': feat_steps
    }


In [ ]:
# Run counterfactuals on a small test sample and save results

# Parameters adjustable for other samples:
SAMPLE_N = 30                 # how many test rows to run counterfactuals for
TARGET_RELS = [0.10, -0.10]   # compute +10% and -10% counterfactuals
MODE = "realistic"            # "realistic" or "full" features defined above in greedy counterfactual
STEP_FRAC = 0.15
MAX_ITERS = 200
VERBOSE = False

# sample deterministic indices from the provided test set
sample_idx = df_cars_test.sample(min(SAMPLE_N, len(df_cars_test)), random_state=0).index.tolist()

perm_imp_small = perm_imp.copy() if ('perm_imp' in globals() and perm_imp is not None) else None

cf_records = []
for idx in tqdm(sample_idx, desc="counterfactuals"):
    inst = df_cars_test.loc[[idx]].copy()
    for tr in TARGET_RELS:
        cf = find_counterfactual_greedy(
            inst, pipe, X_train,
            target_rel=tr,
            mode=MODE,
            step_frac=STEP_FRAC,
            max_iters=MAX_ITERS,
            verbose=VERBOSE,
            perm_importance_series=perm_imp_small
        )
        cf['target_rel'] = tr
        cf_records.append(cf)

# collect into DataFrame and save
rows = []
for r in cf_records:
    rows.append({
        'index': r['index'],
        'target_rel': r['target_rel'],
        'orig_pred': r['original_pred'],
        'final_pred': r['final_pred'],
        'success': r['success'],
        'iters': r['iters'],
        'changes': r['changes']
    })
df_cf_summary = pd.DataFrame(rows).set_index('index')
df_cf_summary.to_csv("open_ended_counterfactual_summary.csv")
print("Saved open_ended_counterfactual_summary.csv — sample of counterfactual attempts")


In [ ]:
# Compact numeric summary results

# Success rates
success_rates = df_cf_summary.groupby("target_rel")["success"].mean().rename("success_rate")
print("Success rates (fraction of tested cars where the algorithm reached the target):")
for idx, v in success_rates.items():
    print(f"  {int(idx*100)}% target: {v:.3f} ({v*100:.1f}%)")
print()

# Which features were changed most often
feature_counter = Counter()
for changes in df_cf_summary["changes"]:
    for f in changes.keys():
        feature_counter[f] += 1
print("Top features changed (count):", feature_counter.most_common())

# Average magnitude of changes
avg_change = {}
for f in feature_counter.keys():
    deltas = [changes[f] for changes in df_cf_summary["changes"] if f in changes]
    if deltas:
        avg_change[f] = float(np.mean(deltas))
print("Average suggested change (approx):", avg_change)


In [ ]:
# Example plots and recommendation table

# helper: basic visualization for a single summary row
def visualize_counterfactual_summary(index, target_rel, df_summary=df_cf_summary, df_test=df_cars_test, pipe=pipe):
    rec = df_summary.loc[index]
    if isinstance(rec, pd.DataFrame):
        rec = rec.loc[rec['target_rel'] == target_rel].iloc[0]
    inst = df_test.loc[[index]]
    orig_pred = float(pipe.predict(inst)[0])
    final_pred = float(rec['final_pred'])
    changes = rec['changes']
    plt.figure(figsize=(6,3.5))
    plt.bar(['Original', f"After changes ({int(target_rel*100)}%)"], [orig_pred, final_pred], color=['#4C72B0', '#55A868'])
    plt.ylabel("Predicted price (GBP)")
    plt.title(f"Row {index} | Success: {rec['success']}")
    plt.tight_layout()
    plt.show()
    print(f"Original: £{orig_pred:,.0f} | After: £{final_pred:,.0f} | Steps: {rec['iters']} | Success: {rec['success']}")
    if changes:
        print("Changes applied (feature -> new value):")
        for k, v in changes.items():
            print(f"  - {k}: {v:,.2f}")
    print("---")

# show two illustrative examples if present (two indexes we use in markdown)
examples_to_show = [100785, 83321]
for idx in examples_to_show:
    if idx in df_cf_summary.index:
        try:
            visualize_counterfactual_summary(idx, 0.10)
        except Exception as e:
            print("visualization error for", idx, ":", e)

# simple textual recommendation column
def simple_recommendation(row):
    if row['success']:
        changes = row['changes']
        if not changes:
            return "No change needed"
        large = any((k == "mpg" and abs(v) > 15) or (k == "miles_per_year" and abs(v) > 2000) or (k == "tax" and abs(v) > 50)
                    for k, v in changes.items())
        return "Consider small repairs" if not large else "Likely unrealistic — manual review"
    else:
        return "Cannot reach target — manual review"

df_readable = df_cf_summary.reset_index().copy()
df_readable['recommendation'] = df_readable.apply(simple_recommendation, axis=1)

# display a clean table
display_cols = ["index", "orig_pred", "final_pred", "target_rel", "success", "iters", "changes", "recommendation"]
display_df = df_readable.loc[:, display_cols].rename(columns={
    "index": "row_id",
    "orig_pred": "Original price (GBP)",
    "final_pred": "Final price after changes (GBP)",
    "target_rel": "Target change",
    "success": "Reached target?",
    "iters": "Search steps",
    "changes": "What changed (feature -> new value)"
})

# format for readability
display_df["Original price (GBP)"] = display_df["Original price (GBP)"].round(0)
display_df["Final price after changes (GBP)"] = display_df["Final price after changes (GBP)"].round(0)
display_df["Target change"] = display_df["Target change"].apply(lambda x: f"{int(x*100)}%")
display_df["Reached target?"] = display_df["Reached target?"].apply(lambda v: "Yes" if v else "No")

display(display_df.head(10))
